In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#imports
import glob
import numpy as np
import torch

In [ ]:
text_path = '/content/gdrive/My Drive/text-generation/text/'
newpath = '/content/gdrive/My Drive/text-generation/newtext/'

for files in glob.glob(text_path +"*.txt"):
    infile = open(files, "r")
    outfile = open(newpath+'new.txt','a+')
    
    for long_sentence in infile:
      long_sentence = long_sentence.encode('ascii', 'ignore').decode('unicode_escape')
      long_sentence = long_sentence.strip()
      outfile.write("%s\n"%long_sentence)
      
    infile.close()
    outfile.close
print("done")



done


In [ ]:
# open text file and read in data as `text`
with open('/content/gdrive/My Drive/text-generation/newtext/new.txt', 'r') as f:
    text = f.read()
    
text = text.encode('ascii', 'ignore').decode('unicode_escape')
text = text.strip()

In [ ]:
text[:100]

In [ ]:
words = set(text.split())
print("total number of unique words",len(words))

total number of unique words 16898


In [ ]:
chars = tuple(set(text.split()))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text.split()])

In [ ]:
encoded[:7]

In [ ]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [ ]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded words in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [ ]:
batches = get_batches(encoded, 32, 100)
x, y = next(batches)

# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[ 7180 14367 12948  1775  5727 11767 16040  2251   986  9590]
 [ 6750 14516  8341 13060 11775  3837 16004  1851  8674  9590]
 [ 1775   101  6702  8929  5776   312 14116 15233 11288 14189]
 [ 8742  7238 14611 15306 10589  7635 10871  3920  9734 11669]
 [12871  4372    43  1727 12128 14814  1183  6406  8896  6684]
 [ 8131 11728  5955  1775 10116 15207  4372 13381  7330  2144]
 [ 2042 14810  9590  1922  2144 10264  1623 11598  7567  9298]
 [14367  8269 14331  5744  2113  7532  1775 11542 16112  6036]
 [  182 16473 13380  1749 12181 16276  4333  4268  4206  3052]
 [14540  1775  3837  9306  7330 15277 12425  7058  3207 14367]]

y
 [[14367 12948  1775  5727 11767 16040  2251   986  9590   670]
 [14516  8341 13060 11775  3837 16004  1851  8674  9590  4467]
 [  101  6702  8929  5776   312 14116 15233 11288 14189 16317]
 [ 7238 14611 15306 10589  7635 10871  3920  9734 11669  8181]
 [ 4372    43  1727 12128 14814  1183  6406  8896  6684  1727]
 [11728  5955  1775 10116 15207  4372 13381  73

In [ ]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [ ]:
from torch import nn
import torch.nn.functional as F

In [ ]:
class WordRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.6, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        self.lstm2 = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        r_output2, hidden2 = self.lstm(x, hidden)
        
        out = self.dropout(r_output2)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden2
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [ ]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.01, clip=5, val_frac=0.2, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.RMSprop(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length))
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [ ]:
# define and print the net
n_hidden=100
n_layers=2

net = WordRNN(chars, n_hidden, n_layers)
print(net)

WordRNN(
  (lstm): LSTM(16898, 100, num_layers=2, batch_first=True, dropout=0.6)
  (lstm2): LSTM(16898, 100, num_layers=2, batch_first=True, dropout=0.6)
  (dropout): Dropout(p=0.6)
  (fc): Linear(in_features=100, out_features=16898, bias=True)
)


In [ ]:
batch_size = 128
seq_length = 100
n_epochs = 300 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.005, print_every=10)

Epoch: 1/300... Step: 10... Loss: 7.8259... Val Loss: 7.5751
Epoch: 1/300... Step: 20... Loss: 7.7047... Val Loss: 7.5007
Epoch: 2/300... Step: 30... Loss: 7.6116... Val Loss: 7.4940
Epoch: 2/300... Step: 40... Loss: 7.5793... Val Loss: 7.4456
Epoch: 3/300... Step: 50... Loss: 7.4886... Val Loss: 7.4355
Epoch: 3/300... Step: 60... Loss: 7.5076... Val Loss: 7.4169
Epoch: 4/300... Step: 70... Loss: 7.4759... Val Loss: 7.3028
Epoch: 4/300... Step: 80... Loss: 7.2742... Val Loss: 7.2101
Epoch: 5/300... Step: 90... Loss: 7.2782... Val Loss: 7.0791
Epoch: 5/300... Step: 100... Loss: 7.0887... Val Loss: 6.9695
Epoch: 6/300... Step: 110... Loss: 7.0853... Val Loss: 7.0078
Epoch: 6/300... Step: 120... Loss: 7.0477... Val Loss: 6.8148
Epoch: 7/300... Step: 130... Loss: 6.8698... Val Loss: 6.7438
Epoch: 7/300... Step: 140... Loss: 6.8711... Val Loss: 6.7096
Epoch: 8/300... Step: 150... Loss: 6.6877... Val Loss: 6.5726
Epoch: 8/300... Step: 160... Loss: 6.8024... Val Loss: 6.5263
Epoch: 9/300... S

In [ ]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch2.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [ ]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        
        x = np.array([[net.char2int[char]]])
        
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [ ]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime.split()]
    h = net.init_hidden(1)
    for ch in prime.split():
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ' '.join(chars)

In [ ]:
print(sample(net, 100, prime='The', top_k=5))

The German life which means TV way you tossing flu hostage Africa Maybe an There's my place in mind, if difference on you a run her Just blame to effectively it. reason just day you dealt of out their work that is less No, but do I'm as this Just remember, To worse, they hope I mean he looks (the stumbled behind of strangers named Q: will a second to play I have what I want to start or Thank those not. So - might consider the age I like you written forever. What that Why might having be than imagine people
